### Question 3. Trip Segmentation Count

During the period of October 1st 2019 (inclusive) and November 1st 2019 (exclusive), how many trips, respectively, happened:

- Up to 1 mile
- In between 1 (exclusive) and 3 miles (inclusive)
- In between 3 (exclusive) and 7 miles (inclusive)
- In between 7 (exclusive) and 10 miles (inclusive)
- Over 10 miles

Answers:

- 104,802; 197,670; 110,612; 27,831; 35,281
- 104,802; 198,924; 109,603; 27,678; 35,189
- 104,793; 201,407; 110,612; 27,831; 35,281
- 104,793; 202,661; 109,603; 27,678; 35,189
- 104,838; 199,013; 109,645; 27,688; 35,202


In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd

# Define variables
start_date = '2019-10-01'
end_date = '2019-11-01'
max_trip_distance = 1

# Create engine
engine = create_engine("postgresql://postgres:postgres@db:5432/ny_taxi")

# SQL query with placeholders for variables
query = text("""
    SELECT COUNT(*)
    FROM green_taxi_data
    WHERE CAST(lpep_dropoff_datetime AS DATE) >= :start_date 
      AND CAST(lpep_dropoff_datetime AS DATE) < :end_date 
      AND CAST(lpep_pickup_datetime AS DATE) >= :start_date 
      AND CAST(lpep_pickup_datetime AS DATE) < :end_date
      AND trip_distance <= :max_trip_distance;
""")

# Execute the query
with engine.connect() as connection:
    result = connection.execute(query, {"start_date": start_date, "end_date": end_date, "max_trip_distance": max_trip_distance})
    for row in result:
        print("Total records: ", row[0])

query_2 = text(
    """
               SELECT COUNT(*)
FROM green_taxi_data
WHERE CAST(lpep_dropoff_datetime as DATE) >= '2019-10-01' AND CAST(lpep_dropoff_datetime as DATE) < '2019-11-01' AND CAST(lpep_pickup_datetime as DATE) >= '2019-10-01' AND CAST(lpep_pickup_datetime as DATE) < '2019-11-01' AND trip_distance > 1 AND trip_distance <= 3;

"""
)

# Close connection
connection.close()

Total records:  104802


In [9]:
# Define distance ranges
distance_ranges = [
    (1, 3),
    (3, 7),
    (7, 10)
]

# Execute queries for each distance range
for min_distance, max_distance in distance_ranges:
    query = text("""
        SELECT COUNT(*)
        FROM green_taxi_data
        WHERE CAST(lpep_dropoff_datetime AS DATE) >= :start_date 
          AND CAST(lpep_dropoff_datetime AS DATE) < :end_date 
          AND CAST(lpep_pickup_datetime AS DATE) >= :start_date 
          AND CAST(lpep_pickup_datetime AS DATE) < :end_date
          AND trip_distance > :min_distance 
          AND trip_distance <= :max_distance;
    """)
    
    with engine.connect() as connection:
        result = connection.execute(query, {"start_date": start_date, "end_date": end_date, "min_distance": min_distance, "max_distance": max_distance})
        for row in result:
            print(f"Total records for distance range ({min_distance}, {max_distance}]: ", row[0])

Total records for distance range (1, 3]:  198924
Total records for distance range (3, 7]:  109603
Total records for distance range (7, 10]:  27678


In [11]:
query_5 = text(
    """
    SELECT COUNT(*)
    FROM green_taxi_data
    WHERE CAST(lpep_dropoff_datetime as DATE) >= :start_date 
      AND CAST(lpep_dropoff_datetime as DATE) < :end_date 
      AND CAST(lpep_pickup_datetime as DATE) >= :start_date 
      AND CAST(lpep_pickup_datetime as DATE) < :end_date 
      AND trip_distance > :max_distance;
    """
)

with engine.connect() as connection:
    result = connection.execute(query_5, {"start_date": start_date, "end_date": end_date, "max_distance": 10})
    for row in result:
        print("Total records for distance range (10, inf): ", row[0])
        
# Close connection
connection.close()

Total records for distance range (10, inf):  35189


### Question 4. Longest Trip for Each Day

Which was the pick-up day with the longest trip distance? Use the pick-up time for your calculations.

**Tip:** For every day, we only care about one single trip with the longest distance.

- 2019-10-11
- 2019-10-24
- 2019-10-26
- 2019-10-31


In [16]:
query_q_4 = text("""
    SELECT 
    CAST(lpep_pickup_datetime AS DATE) AS pickup_date, 
    MAX(trip_distance) AS max_trip_distance
    FROM green_taxi_data 
    WHERE CAST(lpep_pickup_datetime AS DATE) IN 
        ('2019-10-11', '2019-10-24', '2019-10-26', '2019-10-31') 
    GROUP BY CAST(lpep_pickup_datetime AS DATE) 
    ORDER BY max_trip_distance DESC;
""")

with engine.connect() as connection:
    result = connection.execute(query_q_4)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    display(df)
        
        
# Close connection
connection.close()

,pickup_date,max_trip_distance
0,2019-10-31,515.89
1,2019-10-11,95.78
2,2019-10-26,91.56
3,2019-10-24,90.75


### Question 5. Three Biggest Pickup Zones

Which were the top pickup locations with over 13,000 in total_amount (across all trips) for 2019-10-18?

Consider only lpep_pickup_datetime when filtering by date.

- East Harlem North, East Harlem South, Morningside Heights
- East Harlem North, Morningside Heights
- Morningside Heights, Astoria Park, East Harlem South
- Bedford, East Harlem North, Astoria Park

In [17]:
query_q_5 = text("""
    SELECT "PULocationID", zones."Zone", SUM(green_taxi_data.total_amount)
    FROM green_taxi_data
    JOIN zones ON green_taxi_data."PULocationID" = zones."LocationID"
    WHERE CAST(green_taxi_data.lpep_pickup_datetime AS DATE) = '2019-10-18'
    GROUP BY "PULocationID", zones."Zone"
    HAVING SUM(green_taxi_data.total_amount) > 13000;
""")

with engine.connect() as connection:
    result = connection.execute(query_q_5)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    display(df)

# Close connection
connection.close()

,PULocationID,Zone,sum
0,74,East Harlem North,18686.68
1,75,East Harlem South,16797.26
2,166,Morningside Heights,13029.79


### Question 6. Largest tip

For the passengers picked up in October 2019 in the zone named "East Harlem North," which was the drop-off zone that had the largest tip?

Note: it's tip, not trip.

We need the name of the zone, not the ID.

- Yorkville West
- JFK Airport
- East Harlem North
- East Harlem South

In [18]:
query_q_6 = text(
    """
    SELECT 
        z2."Zone" as "DOZone",
        MAX(green_taxi_data.tip_amount) AS max_tip
    FROM green_taxi_data
    INNER JOIN zones z1 ON z1."LocationID" = green_taxi_data."PULocationID"
    INNER JOIN zones z2 ON z2."LocationID" = green_taxi_data."DOLocationID"
    WHERE z1."Zone" = 'East Harlem North'
    GROUP BY z2."LocationID", z2."Zone"
    ORDER BY max_tip DESC
    LIMIT 1;
    """
)

with engine.connect() as connection:
    result = connection.execute(query_q_6)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    display(df)

# Close connection
connection.close()

,DOZone,max_tip
0,JFK Airport,87.3
